In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Here we define the dataset depending on kitti sequence!

In [2]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
SEQUENCE_NUM = 7

ind_start = 0
ind_end = 1100
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = 'pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'test_data/'
if os.path.exists(out_folder_ncuts):
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [3]:
if os.path.exists(out_folder + 'all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)

This cell can be ignored after first run as outputs are stored 

In [4]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz',
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )


In [5]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd')
print(pcd_ground_minor)
kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz') as data :
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

PointCloud with 11580178 points.


In [6]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

"\npcd_new = o3d.geometry.PointCloud()\npts_num = 1000000\npcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])\n\nmap_labelled = color_pcd_by_labels(pcd_new,                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))\n\no3d.visualization.draw_geometries([map_labelled])\n#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)\n"

Now we subsample the poses based on a voxel_size

In [7]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

Now we can split the point cloud into chunks based on a tbd chunk_size

In [8]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

Downsampled from (541161, 3) to (7776, 3) points (non-ground)
Downsampled from (243917, 3) to (5294, 3) points (ground)
Downsampled from (454490, 3) to (7216, 3) points (non-ground)
Downsampled from (262770, 3) to (4499, 3) points (ground)
Downsampled from (479361, 3) to (7341, 3) points (non-ground)
Downsampled from (304983, 3) to (6061, 3) points (ground)
Downsampled from (479273, 3) to (6272, 3) points (non-ground)
Downsampled from (246003, 3) to (4418, 3) points (ground)
Downsampled from (473079, 3) to (8385, 3) points (non-ground)
Downsampled from (221993, 3) to (4446, 3) points (ground)
Downsampled from (488393, 3) to (9063, 3) points (non-ground)
Downsampled from (220415, 3) to (5078, 3) points (ground)
Downsampled from (545707, 3) to (8986, 3) points (non-ground)
Downsampled from (292301, 3) to (4209, 3) points (ground)
Downsampled from (409731, 3) to (6703, 3) points (non-ground)
Downsampled from (441195, 3) to (4976, 3) points (ground)
Downsampled from (300320, 3) to (4742, 3

In [9]:
import sklearn
from sklearn.cluster import DBSCAN

def DBSCAN_clustering_logic(cur_pcd, pcd_all, eps=0.3, min_samples=10):
    """
    Perform DBSCAN clustering on the point cloud data.

    :param cur_pcd: Current point cloud for clustering.
    :param pcd_all: All point cloud data.
    :param eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    :param min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
    :return: Cluster labels for each point in the point cloud.
    """
    not_road_points = np.asarray(cur_pcd.points)
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(not_road_points)
    #clustering = HDBSCAN(min_cluster_size=100).fit(not_road_points)
    labels_not_road = clustering.labels_
    colors_gen = generate_random_colors(500)
    
    # Reproject cluster labels to the original point cloud size
    cluster_labels = np.ones((len(pcd_all.points), 1)) * -1
    labels_non_ground = kDTree_1NN_feature_reprojection(cluster_labels, pcd_all, labels_not_road.reshape(-1,1), cur_pcd)
    colors = np.zeros((labels_non_ground.shape[0],3))
    unique_labels = list(np.unique(labels_non_ground))
    for j in unique_labels:
            cur_idcs = np.where(labels_non_ground == j)[0]
            
            colors[cur_idcs] = np.array(colors_gen[unique_labels.index(j)])
    pcd_all.colors = o3d.utility.Vector3dVector(colors / 255.)
    return pcd_all

def dbscan_clustering(pcd_chunks_major_downsampled, pcds, center_ids,ground_clouds):
    labels_clustering = []
    for i in range(len(pcd_chunks_major_downsampled)):
        cur_pcd = pcd_chunks_major_downsampled[i]
        pcd_all = pcds[i]
        ground_cloud = ground_clouds[i]
        cluster_labels = DBSCAN_clustering_logic(cur_pcd, pcd_all)  # Implement your DBSCAN logic here
        ground_labels = np.ones((np.asarray(ground_cloud.points).shape[0],1)) * -1
        cluster_labels = np.concatenate((cluster_labels,ground_labels),0)
        labels_clustering.append(cluster_labels)
        
    return labels_clustering


In [10]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored

In [11]:
alpha = 1.0
theta = 1.0
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0
ncuts_threshold = 0.01
        


out_kitti = out_chunks + 'out_kitti/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        
out_kitti_instance = out_chunks + 'out_kitti_instance/'
if os.path.exists(out_kitti_instance) == True : 
        shutil.rmtree(out_kitti_instance)
os.makedirs(out_kitti_instance)

out_kitti_semantic = out_chunks + 'out_kitti_semantic/'
if os.path.exists(out_kitti_semantic) == True : 
        shutil.rmtree(out_kitti_semantic)
os.makedirs(out_kitti_semantic)

limit = -1 ##use this for experiments to run limit chunks numberss


patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
semantics = np.hstack((kitti_labels_orig['seg_nonground'],kitti_labels_orig['seg_ground']))

instances = np.hstack((kitti_labels_orig['instance_nonground'],kitti_labels_orig['instance_ground']))
                

for sequence in range(len(center_ids)):
        if NCUT_ground == False : 
                
                merged_chunk,file_name, pcd_chunk, pcd_chunk_ground,inliers, inliers_ground = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold)

                
                

                seg_ground = kitti_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                inst_ground = kitti_labels['ground']['instance'][sequence][inliers][inliers_ground]
                
                
                
                file_name = str(center_ids[sequence]).zfill(6) + '.pcd'
                #o3d.io.write_point_cloud(file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                
                #pcd_dbscan = DBSCAN_clustering_logic(pcd_nonground_chunks_major_downsampling[sequence],
                #                                pcd_nonground_chunks[sequence],
                #                                eps=0.6, min_samples=10)
                
                
                
                
                kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['semantic'][sequence].reshape(-1,),
                                        colors=colors,gt_labels=semantics
                                        )
                
                kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),
                                        colors=colors,gt_labels=instances)
                                        
                kitti_chunk_semantic_ground = color_pcd_by_labels(pcd_chunk_ground,seg_ground.reshape(-1,),
                                        colors=colors,gt_labels=semantics)
                #o3d.visualization.draw_geometries([kitti_chunk_instance + kitti_chunk_instance_ground])
                #o3d.visualization.draw_geometries([kitti_chunk + kitti_chunk_semantic_ground])
                
                
                
        
                #print(kitti_labels['ground']['semantic'][sequence].reshape(-1,).shape)
                #print(kitti_labels_orig['seg_ground'].shape)
                #print(kitti_chunk_semantic_ground)
                
                o3d.io.write_point_cloud(out_folder_ncuts + file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                o3d.io.write_point_cloud(out_kitti_instance + file_name, kitti_chunk_instance + kitti_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)
                o3d.io.write_point_cloud(out_kitti_semantic + file_name, kitti_chunk + kitti_chunk_semantic_ground, write_ascii=False, compressed=False, print_progress=False)


        
        
        

Start of sequence 0
7776 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 31 cut regions
Ratio of points in top 3 groups: 0.6132973251028807
Start of sequence 1
7216 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 22 cut regions
Ratio of points in top 3 groups: 0.8264966740576497
Start of sequence 2
7341 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 23 cut regions
Ratio of points in top 3 groups: 0.7618853017300096
Start of sequence 3
6272 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 18 cut regions
Ratio of points in top 3 groups: 0.7975127551020408
Start of sequence 4
8385 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 21 cut 

In [ ]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds




def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

In [ ]:
out_dbscan = out_chunks + 'out_dbscan/'
out_kitti = out_chunks + 'out_kitti/'
out_kitti_instance = out_chunks + 'out_kitti_instance/'
out_kitti_semantic = out_chunks + 'out_kitti_semantic/'

point_clouds = get_merge_pcds(out_folder_ncuts)[:-1]
#point_clouds_kitti = get_merge_pcds(out_kitti)[:-1]
#point_clouds_kitti_instances = get_merge_pcds(out_kitti_instance)
#point_clouds_kitti_semantic = get_merge_pcds(out_kitti_semantic)[:-1]
merge = merge_chunks_unite_instances(point_clouds)


#merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)

#merge_kitti = merge_unite_gt(point_clouds_kitti)
#merge_kitti_instance = merge_unite_gt(point_clouds_kitti_instances)
#merge_kitti_semantic = merge_unite_gt(point_clouds_kitti_semantic)

#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_semantic7.pcd", merge_kitti_semantic, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "tarl_spatial_test.pcd", merge, write_ascii=False, compressed=False, print_progress=False)
#o3d.io.write_point_cloud(out_folder + "merge_part_kitti_instance7.pcd", merge_kitti_instance, write_ascii=False, compressed=False, print_progress=False)


['000066.pcd', '000093.pcd', '000125.pcd', '000164.pcd', '000193.pcd', '000220.pcd', '000249.pcd', '000278.pcd', '000336.pcd', '000368.pcd', '000391.pcd', '000415.pcd', '000441.pcd', '000483.pcd', '000515.pcd', '000540.pcd', '000565.pcd', '000587.pcd', '000610.pcd', '000650.pcd', '000774.pcd', '000794.pcd', '000813.pcd', '000833.pcd', '000854.pcd', '000877.pcd', '000915.pcd', '000949.pcd', '000982.pcd', '001048.pcd']


True

In [ ]:
'''
optional cell for loading stored files
import open3d as o3d

out_folder = 'pcd_preprocessed/'
merge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')
merge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')
merge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')
merge_exp2 = o3d.io.read_point_cloud(out_folder + 'merge_exp2.pcd')

print(merge)
print(merge_kitti)
print(merge_exp2)
'''

"\noptional cell for loading stored files\nimport open3d as o3d\n\nout_folder = 'pcd_preprocessed/'\nmerge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')\nmerge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')\nmerge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')\nmerge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance.pcd')\nmerge_exp2 = o3d.io.read_point_cloud(out_folder + 'merge_exp2.pcd')\n\nprint(merge)\nprint(merge_kitti)\nprint(merge_exp2)\n"

In [ ]:

unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance7.pcd')
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels




new_ncuts_labels = remove_semantics(labels_kitti,labels_ncuts)


metrics_ncuts = Metrics(name='ncuts')

metrics_ncuts.update_stats(new_ncuts_labels,labels_kitti)

#merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
#o3d.visualization.draw_geometries([merge_vis])


Metrics for file ncuts
{'panoptic': 0.6445567981103508, 'precision': 0.85, 'recall': 0.6296296296296297, 'fScore': 0.723404255319149, 'usr': 0.1125, 'osr': 0.009259259259259259, 'noise': 0.075, 'missed': 0.2037037037037037, 'mean': 0.8910049856231319}
lstq value :  0.6125271870457567
Average Precision @ 0.25 0.6345500434166992
Average Precision @ 0.5 0.5227843433234283
Average Precision @ 0.55 0.5015184590439679
Average Precision @ 0.6 0.4672227293847144
Average Precision @ 0.65 0.4390970624207204
Average Precision @ 0.7 0.40320934245404133
Average Precision @ 0.75 0.38019380860779267
Average Precision @ 0.85 0.35179566937293477
Average Precision @ 0.8 0.36140010442163195
Average Precision @ 0.9 0.33701263870176507
Average Precision @ 0.95 0.09224409028491193
AP @ 0.25 63.455
AP @ 0.5 52
AP @ [0.5:0.95] 38.565


AttributeError: 'Metrics' object has no attribute 'compute_all_aps'

In [ ]:
o3d.visualization.draw_geometries([merge])